# Fitting keras functional API model with tf.data.DataSet

### Imports

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

print('numpy version: {}'.format(np.__version__))
print('keras version: {}'.format(keras.__version__))
print('tensorflow version: {}'.format(tf.__version__))

numpy version: 1.21.4
keras version: 2.6.0
tensorflow version: 2.6.0


### GPU and Environment

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "6"
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

### Psuedo Data

In [12]:
X = np.random.uniform(size=(1000,75))
Y = np.random.uniform(size=(1000))

data = tf.data.Dataset.from_tensor_slices((X, Y))
print(data.cardinality().numpy())

1000


In [13]:
data.batch(batch_size=100, drop_remainder=True)

<BatchDataset shapes: ((100, 75), (100,)), types: (tf.float64, tf.float64)>

### Make Model

In [14]:
def API_Model(input_shape, name="test_model"):
    
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(1)(inputs)
    outputs = layers.Activation('relu')(x)
    
    return keras.Model(inputs=inputs, outputs=outputs, name=name)

api_model = API_Model(input_shape=(X.shape[1],))
api_model.compile()
api_model.summary()

Model: "test_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 75)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 76        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________


In [15]:
api_model.fit(data, epochs=10)

Epoch 1/10


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /usr/local/lib/python3.8/dist-pack

# Protobuf Error and Placeholders

# Custom Layers and Eager Execution